In [1]:
import pandas as pd;
import numpy as np
import sagemaker
from sagemaker import get_execution_role
import boto3
import re

ModuleNotFoundError: No module named 'sagemaker'

In [ ]:
bucket_name='awssagemakerml77'
train_data= r'costofcars/training/'
val_data=r'costofcars/validation/'
s3_model_output_location=r's3://{0}/costofcars/model'.format(bucket_name)
s3_training_file_location=r's3://{0}/{1}'.format(bucket_name,train_data)
s3_validation_file_location=r's3://{0}/{1}'.format(bucket_name,val_data)

In [ ]:
def write_to_s3(filename,bucket,key):
    with open(filename,'rb') as f:
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [ ]:
write_to_s3('train.csv',bucket_name,train_data+'train.csv')
write_to_s3('test.csv',bucket_name,val_data+'test.csv')
role=get_execution_role()

In [ ]:
sess=sagemaker.Session()

In [ ]:
container=sagemaker.amazon.amazon_estimator.get_image_uri(sess.boto_region_name,"xgboost","latest")

In [ ]:
estimator=sagemaker.estimator.Estimator(container,role,train_instance_count=1,train_instance_type='ml.m4.xlarge',
                                        output_path=s3_model_output_location,sagemaker_session=sess,
                                       base_job_name='v1-xgboost-costofcars')

In [ ]:
estimator.set_hyperparameters(max_depth=3,num_round=500)

In [ ]:
training_input_config=sagemaker.session.s3_input(s3_data=s3_training_file_location,content_type='csv',
                                                s3_data_type='S3Prefix')
validation_input_config=sagemaker.session.s3_input(s3_data=s3_validation_file_location,content_type='csv',
                                                s3_data_type='S3Prefix')

In [ ]:
data_channels={'train':training_input_config,'validation':validation_input_config}

In [ ]:
estimator.fit(data_channels)

In [ ]:
predictor=estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge',endpoint_name='v3-xgboost-bcancer')

In [ ]:
from sagemaker.predictor import csv_serializer,json_deserializer
predictor.serializer=csv_serializer
predictor.deserializer=None
predictor.content_type='test/csv'

In [ ]:
endpoint_name='v3-xgboost-bcancer'
predictor=sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name)

In [ ]:
df_test=pd.read_csv('test.csv',header=None)

In [ ]:
df_test.head()

In [ ]:
arr_test=df_test[df_test.columns[1:]].values

In [ ]:
arr_test.shape

In [ ]:
arr_test[0]

In [ ]:
result=predictor.predict(arr_test[0])